In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

2025-03-06 18:07:00,804 INFO: Initializing external client
2025-03-06 18:07:00,804 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 18:07:01,726 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214648
Fetching data from 2025-02-05 23:07:00.803135+00:00 to 2025-03-06 22:07:00.803135+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.85s) 
Skipping location_id 156: Not enough data to create even one window.
Skipping location_id 84: Not enough data to create even one window.
Skipping location_id 44: Not enough data to create even one window.
Skipping location_id 109: Not enough data to create even one window.
Skipping location_id 204: Not enough data to create even one window.
Skipping location_id 5: Not enough data to create even one window.


In [4]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-06 18:07:12,064 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 18:07:12,066 INFO: Initializing external client
2025-03-06 18:07:12,066 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 18:07:12,807 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214648

In [5]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [6]:
predictions["pickup_hour"] = current_date.ceil('h')
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,189,0.0,2025-03-07 00:00:00+00:00
1,197,0.0,2025-03-07 00:00:00+00:00
2,247,0.0,2025-03-07 00:00:00+00:00
3,179,0.0,2025-03-07 00:00:00+00:00
4,21,0.0,2025-03-07 00:00:00+00:00
...,...,...,...
247,150,0.0,2025-03-07 00:00:00+00:00
248,183,0.0,2025-03-07 00:00:00+00:00
249,254,0.0,2025-03-07 00:00:00+00:00
250,163,185.0,2025-03-07 00:00:00+00:00


In [7]:
from src.inference import get_feature_store

feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1,
    description="Predictions from LGBM Model",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour",
)

2025-03-06 18:07:23,422 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 18:07:23,426 INFO: Initializing external client
2025-03-06 18:07:23,426 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 18:07:24,204 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214648


In [8]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1214648/fs/1202279/fg/1410378


Uploading Dataframe: 100.00% |████████████████████████████████████████████████████████| Rows 252/252 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: taxi_hourly_model_prediction_v5_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1214648/jobs/named/taxi_hourly_model_prediction_v5_1_offline_fg_materialization/executions


(Job('taxi_hourly_model_prediction_v5_1_offline_fg_materialization', 'SPARK'),
 None)